## GenAI Project
Develope a question answer chatbot for a website information
We will use agile description website (https://asana.com/resources/agile-methodology) to query for the detials

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GOOGLE_API_KEY"]=os.getenv("google_api_key")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("langchain_api_key")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("langchain_project")

In [16]:
# Data ingestion - scrape from website
from langchain_community.document_loaders import WebBaseLoader

In [6]:
loader=WebBaseLoader("https://asana.com/resources/agile-methodology")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://asana.com/resources/agile-methodology', 'title': 'What Is Agile Methodology? (A Beginner’s Guide) [2025] • Asana', 'description': 'Agile methodology is a framework that breaks projects down into several dynamic phases, commonly known as sprints. Learn more. ', 'language': 'en'}, page_content='\n\nWhat Is Agile Methodology? (A Beginner’s Guide) [2025] • AsanaJoin us: Learn how to build a trusted AI strategy to support your company\'s intelligent transformation, featuring ForresterRegister nowx iconAsana HomeProduct chevron-down iconSolutions chevron-down iconResources chevron-down iconPricingChoose your preferred languageBahasa IndonesiaDeutschEnglishEspañolFrançaisItaliano日本語한국어NederlandsPolskiPortuguêsРусскийSvenska繁體中文Contact salesLog InGet startedProductchevron-down iconPLATFORMProduct overviewAll featuresLatest feature releaseApp integrationsCAPABILITIESproject iconProject managementgoal iconGoals and reportingasana-intelligence iconAsana AIwo

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [9]:
documents

[Document(metadata={'source': 'https://asana.com/resources/agile-methodology', 'title': 'What Is Agile Methodology? (A Beginner’s Guide) [2025] • Asana', 'description': 'Agile methodology is a framework that breaks projects down into several dynamic phases, commonly known as sprints. Learn more. ', 'language': 'en'}, page_content="What Is Agile Methodology? (A Beginner’s Guide) [2025] • AsanaJoin us: Learn how to build a trusted AI strategy to support your company's intelligent transformation, featuring ForresterRegister nowx iconAsana HomeProduct chevron-down iconSolutions chevron-down iconResources chevron-down iconPricingChoose your preferred languageBahasa IndonesiaDeutschEnglishEspañolFrançaisItaliano日本語한국어NederlandsPolskiPortuguêsРусскийSvenska繁體中文Contact salesLog InGet startedProductchevron-down iconPLATFORMProduct overviewAll featuresLatest feature releaseApp integrationsCAPABILITIESproject iconProject managementgoal iconGoals and reportingasana-intelligence iconAsana AIworkflo

In [17]:
len(documents)

25

In [18]:
# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", task_type="retrieval_document")

In [19]:
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(documents,embeddings)
db

In [33]:
# Retrieval
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [34]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0.3,
    max_retries=2
)

In [35]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the provided context. For null context return output as, I do not know. :
    <context>
    {context}
    </context>
    """
)

In [36]:
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context. For null context return output as, I do not know. :\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-001', google_api_key=SecretStr('**********'), temperature=0.3, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001E3F75F0550>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [38]:
from langchain.chains import create_retrieval_chain

In [39]:
retriever=db.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E3F54BC070>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context. For null context return output as, I do not know. :\n    <context>\n  

In [40]:
response=retrieval_chain.invoke({"input":"What are the 4 pillars of Agile?"})
response['answer']

'Agile methodology is a project management framework that breaks projects down into several dynamic phases, commonly known as sprints. It is an iterative methodology where teams reflect after each sprint to identify areas for improvement and adjust their strategy accordingly. The Agile Manifesto outlines the four values and 12 principles for Agile software development. The four main values of Agile project management are:\n\n*   Individuals over processes and tools\n*   Working software over comprehensive documentation\n*   Customer collaboration over contract negotiation\n*   Responding to change over following a plan\n\nThe 12 Agile principles are:\n\n*   Satisfy customers through early, continuous improvement and delivery.\n*   Welcome changing requirements, even late in the project.\n*   Deliver value frequently.'

'Agile methodology is a project management framework that breaks projects down into several dynamic phases, commonly known as sprints. It is an iterative methodology where teams reflect after each sprint to identify areas for improvement and adjust their strategy accordingly. The Agile Manifesto outlines the four values and 12 principles for Agile software development. The four main values of Agile project management are:\n\n*   Individuals over processes and tools\n*   Working software over comprehensive documentation\n*   Customer collaboration over contract negotiation\n*   Responding to change over following a plan\n\nThe 12 Agile principles are:\n\n*   Satisfy customers through early, continuous improvement and delivery.\n*   Welcome changing requirements, even late in the project.\n*   Deliver value frequently.'

In [41]:
response

{'input': 'What are the 4 pillars of Agile?',
 'context': [Document(id='afb96c3e-1d88-437d-9842-6eed4475eca5', metadata={'source': 'https://asana.com/resources/agile-methodology', 'title': 'What Is Agile Methodology? (A Beginner’s Guide) [2025] • Asana', 'description': 'Agile methodology is a framework that breaks projects down into several dynamic phases, commonly known as sprints. Learn more. ', 'language': 'en'}, page_content='is the Agile methodology?Agile methodology is a project management framework that breaks projects down into several dynamic phases, commonly known as sprints.\xa0The Agile framework is an iterative methodology. After every sprint, teams reflect and look back to see if there was anything that could be improved so they can adjust their strategy for the next sprint.What is the Agile Manifesto?The Manifesto for Agile Software Development is a document that details the four values and 12 principles for Agile software development. It was published in February 2001 b

In [44]:
response=retrieval_chain.invoke({"input":"What are the benefits of Agile?"})
response['answer']

'Here are a few other reasons why teams use Agile:\n\n*   Agile methods are adaptable\n*   Agile fosters collaborative teamwork\n*   Goal of software development projects is a working product\n*   Maintain a sustainable working pace\n*   Continuous excellence enhances agility\n*   Simplicity is essential\n*   Self-organizing teams generate the most value\n*   Regularly reflect and adjust your way of work to improve'

In [45]:
response

{'input': 'What are the benefits of Agile?',
 'context': [Document(id='69eebd99-32b1-482f-87fc-6056c1c2f863', metadata={'source': 'https://asana.com/resources/agile-methodology', 'title': 'What Is Agile Methodology? (A Beginner’s Guide) [2025] • Asana', 'description': 'Agile methodology is a framework that breaks projects down into several dynamic phases, commonly known as sprints. Learn more. ', 'language': 'en'}, page_content="Here are a few other reasons why teams use Agile:Agile methods are adaptableThere's a reason why they call it the Agile methodology. One of the main benefits of using Agile processes in software development is the ability to shift strategies quickly, without disrupting the flow of a project.\xa0Because phases in the traditional waterfall method flow into one another, shifting strategies is challenging and can disrupt the rest of the project roadmap. Since software development is a much more adaptable field, project managing rapid changes in the traditional sens